# Using Rekognition for Video

In [1]:
# initialise Notebook
import imageio
imageio.plugins.ffmpeg.download()
import boto3
from IPython.display import HTML, display
from PIL import Image
from pprint import pprint
from io import BytesIO
import moviepy.editor as mpy
import os
import uuid
import io

YOUR_ACCESS_KEY='###########' #Put your account's ACCESS_KEY
YOUR_SECRET_KEY='###########################' #Put your account's SECRET_ACCESS_KEY

In [2]:
#Initialize the clients
rekognition = boto3.client('rekognition', region_name='us-west-2',
                           aws_access_key_id=YOUR_ACCESS_KEY,aws_secret_access_key=YOUR_SECRET_KEY)
dynamodb = boto3.resource('dynamodb', region_name='us-west-2',
                           aws_access_key_id=YOUR_ACCESS_KEY,aws_secret_access_key=YOUR_SECRET_KEY)
s3 = boto3.client('s3', region_name='us-west-2',
                           aws_access_key_id=YOUR_ACCESS_KEY,aws_secret_access_key=YOUR_SECRET_KEY)

# Build the index of faces and create your meta-store

Before we start identifying personalities from a video, we need to build the Rekognition index of the faces for those personalities and then create our metastore in DynamoDB with those indices.

For this demo, download the jpg images from this repository and upload them to appropriate S3 bucket

In [14]:
%%bash
aws rekognition create-collection --collection-id personalities --region us-west-2

aws:rekognition:us-west-2:681445445985:collection/personalities	200


In [19]:
%%bash
aws dynamodb create-table --table-name rekognition_personalities --attribute-definitions AttributeName=Id,AttributeType=S --key-schema AttributeName=Id,KeyType=HASH --provisioned-throughput ReadCapacityUnits=5,WriteCapacityUnits=5

TABLEDESCRIPTION	1504781722.43	0	arn:aws:dynamodb:us-west-2:681445445985:table/rekognition_personalities	rekognition_personalities	0	CREATING
ATTRIBUTEDEFINITIONS	Id	S
KEYSCHEMA	Id	HASH
PROVISIONEDTHROUGHPUT	0	5	5


In [7]:
s3_image_bucket = 'ai-demo-in'
s3_folder_name = "sample_iamges/for_rekog_demo/"

faces = [
    {"objectId":s3_folder_name+"lionel-messi-1.jpg","fullName":"Lionel Messi"},
    {"objectId":s3_folder_name+"lionel-messi-2.jpg","fullName":"Lionel Messi"},
    {"objectId":s3_folder_name+"lionel-messi-3.jpg","fullName":"Lionel Messi"},
    {"objectId":s3_folder_name+"lionel-messi-4.jpg","fullName":"Lionel Messi"},
    {"objectId":s3_folder_name+"lionel-messi-5.jpg","fullName":"Lionel Messi"}    
]

face_table = dynamodb.Table('rekognition_personalities')

def populate_lookup_table(fullName,perId):
    print ('Populating DynamoDB index for ' + fullName + ' as ' + perId)
    response = face_table.put_item(
                Item={
                      'Id': perId,
                      'FullName': fullName,
                      }
                )
    return response
        
def populate_rekognition_collection(objectId):
    print ('Indexing ', objectId)
    ret = rekognition.index_faces(
    CollectionId=rekognition_face_index,
    Image={
                'S3Object': {
                    'Bucket': s3_image_bucket,
                    'Name':  objectId
                }
            }
        )
    if ret['ResponseMetadata']['HTTPStatusCode'] == 200:
        return ret['FaceRecords'][0]['Face']['FaceId']
    else:
        return False
    

def populate_index(all_faces,rekognition_face_index):
    for f in all_faces:
        ret = populate_rekognition_collection(f['objectId'])
        if ret == False:
            print ('some error')
        else:
            ret = populate_lookup_table(f['fullName'],ret)
            pprint (ret['ResponseMetadata']['HTTPStatusCode'])
        
              
    print ('done')

    
rekognition_face_index = 'personalities' 
populate_index(faces,rekognition_face_index)

('Indexing ', 'sample_iamges/for_rekog_demo/lionel-messi-1.jpg')
Populating DynamoDB index for Lionel Messi as 6c5cee3b-e088-5e59-ae50-7cdf32b0495d
200
('Indexing ', 'sample_iamges/for_rekog_demo/lionel-messi-2.jpg')
Populating DynamoDB index for Lionel Messi as 9387ed04-48c9-54d8-9b23-5a15f61c2fb8
200
('Indexing ', 'sample_iamges/for_rekog_demo/lionel-messi-3.jpg')
Populating DynamoDB index for Lionel Messi as 3570f1c5-6909-51a0-bdac-f32f84833b71
200
('Indexing ', 'sample_iamges/for_rekog_demo/lionel-messi-4.jpg')
Populating DynamoDB index for Lionel Messi as bbb479d3-f229-52c8-a451-25c83bd15136
200
('Indexing ', 'sample_iamges/for_rekog_demo/lionel-messi-5.jpg')
Populating DynamoDB index for Lionel Messi as 320a67ad-b997-5670-bbd2-bbbe7e47f26f
200
done


In [3]:
# Detect face boxes using Rekognition

def face_detection(image_input):
    response = rekognition.detect_faces(
        Image={
            'Bytes':image_input
        }                                        
        )
    
    all_faces=response['FaceDetails']
    
    # initialise list object 
    boxes = []
  
    # populate list for each face
    for face in all_faces:
        boxes.append (face['BoundingBox'])
    return boxes

# apply face boxes to image and store coordinates for later processing
def get_coordinates(face_boxes, image_width, image_height):
    # initialise list object 
    coordinates = [] 
    for box in face_boxes:
        x1 = int(box['Left'] * image_width)-5
        y1 = int(box['Top'] * image_height)-5
        x2 = int(box['Left'] * image_width + box['Width'] * image_width)+5
        y2 = int(box['Top'] * image_height + box['Height']  * image_height)+5
        if x1 < 0 : x1=0
        if y1 < 0 : y1=0
        if x2 < 0 : x2=image_width
        if y2 < 0 : y2=image_height 
            
        coordinates.append((x1,y1,x2,y2))
        
    return coordinates

# Search faces in index

def get_face_data(image_binary):
    
    try:
        response = rekognition.search_faces_by_image(
            CollectionId='personalities',
            Image={
                'Bytes':image_binary
            }                                       
            )
    
        if len(response['FaceMatches']) > 0:
            #for o in response['FaceMatches']:
             #   print (o['Face']['FaceId'],o['Face']['Confidence'],get_name(o['Face']['FaceId']))
            return response['FaceMatches'][0]['Face']['FaceId'],response['FaceMatches'][0]['Face']['Confidence']
        else:
            return ('no match detected',0)
    except Exception as e:
        #print (e)
        return ('no face detected',0)
    
def get_name(perId):
    response = face_table.get_item(
            Key={
                'Id': perId
            },
            AttributesToGet=['FullName']                  
        )
    
    if 'Item' in response:
        print('The video features ', response['Item']['FullName'])
        return (response['Item']['FullName'])
    else:
        return ('The video has known faces but there is no match found in personalities metadata')


# Slicing the Video into Frames

Download the video "Linonel Messi.mp4" into the same directory

In [9]:
key = 'Lionel Messi.mp4' #Name of the video file
uploadDir = '/tmp/uploads/' # local dir where the sliced frames will be saved
    
# download object from S3 and instantiate clip object
clip_location = key
clip = mpy.VideoFileClip(clip_location)
clip_lenghts = clip.duration
    
# slice frames every x second
freqOfSlicing = 2

frames = []
fUNID = str(uuid.uuid4())
cut = 1
frame = 1
while True:
    print ('cutting at ' + str(cut) + 'seconds.')
    clip.save_frame( uploadDir + str(frame) +'-'+ fUNID +'.jpg',cut)
    frames.append (uploadDir + str(frame) +'-'+ fUNID +'.jpg')
    cut += freqOfSlicing
    frame +=1 
    if cut > clip_lenghts:
        break

cutting at 1seconds.
cutting at 3seconds.
cutting at 5seconds.
cutting at 7seconds.
cutting at 9seconds.
cutting at 11seconds.
cutting at 13seconds.
cutting at 15seconds.
cutting at 17seconds.
cutting at 19seconds.
cutting at 21seconds.
cutting at 23seconds.
cutting at 25seconds.
cutting at 27seconds.
cutting at 29seconds.
cutting at 31seconds.
cutting at 33seconds.
cutting at 35seconds.
cutting at 37seconds.
cutting at 39seconds.
cutting at 41seconds.
cutting at 43seconds.
cutting at 45seconds.
cutting at 47seconds.
cutting at 49seconds.
cutting at 51seconds.


# Identify personalities in the video

In [10]:
# detect people in video clip

def get_Faces(image_path):
    # load image into byte stream
    image_bin=Image.open(image_path)
    stream = io.BytesIO()
    image_bin.save(stream,format="JPEG")
    image_binary = stream.getvalue()
    
    # Detect face boxes using Rekognition
    boxes = face_detection(image_binary)
    width, height = image_bin.size
    xys = get_coordinates(boxes,width, height)
    
    # iterate through list of boxes and detect individual face
    # initialise list object 
    ret = []
    for box in xys:
           
        image_crop = image_bin.crop(box)
        
        # add cropped image to temporary stream
        stream2 = io.BytesIO() 
        image_crop.save(stream2,format="JPEG")
        image_region_binary = stream2.getvalue()    
        stream2.close()
        
        try:
            # get external ID name of cropped image
            retdata=get_face_data(image_region_binary),box
            ret.append (get_name(retdata[0][0])) 
        except Exception as e:
            ret.append ((('no face',0),box))
            pass
    

print('Searching'),
for i in frames:
    #print ('processing: ', i)
    print('.'),
    get_Faces(i)
print ('done')

Searching . . . . . . . . . . . . . . . . . . . . . . ('The video features ', u'Lionel Messi')
. . . . done


# Identifying objects in the Video

In [11]:
# get labels for video clip

weight_labels = []
all_labels =[]
    
def get_Labels(image_path):
    # load image into byte stream
    image_bin=Image.open(image_path)
    stream = io.BytesIO()
    image_bin.save(stream,format="JPEG")
    image_binary = stream.getvalue()
    
    # retrieve labels from Rekognition service
    ret = rekognition.detect_labels(
        Image={'Bytes':image_binary},
        MaxLabels=5,
    )
    
    weight_labels.append((image_path,ret['Labels']))
    
    for i in ret['Labels']:
        try:
            all_labels.index(i['Name'])
            continue
        except: 
            if i['Confidence'] > 85:
                pprint (i)

    for l in ret['Labels']:
        all_labels.append(l['Name'])        

list = []
for i in frames:
    #pprint (i)
    get_Labels(i)

{u'Confidence': 98.28312683105469, u'Name': u'People'}
{u'Confidence': 95.06742095947266, u'Name': u'Huddle'}
{u'Confidence': 95.06742095947266, u'Name': u'Speech'}
{u'Confidence': 88.39244842529297, u'Name': u'Alcohol'}
{u'Confidence': 88.39244842529297, u'Name': u'Beverage'}
{u'Confidence': 88.39244842529297, u'Name': u'Liquor'}
{u'Confidence': 86.78985595703125, u'Name': u'Plant'}
{u'Confidence': 86.78985595703125, u'Name': u'Potted Plant'}
